In [1]:
import gzip
from tqdm import tqdm
#将first_jump作为第二跳的起点，保存在second_initial中
second_initial = []
with gzip.open('first_jump.gz','rb') as f:
    for line in tqdm(f, desc="Processing lines", unit="line"):
        second_initial.append(line.strip().decode())
print(len(second_initial))
#开始第二跳
str_flag="<http://rdf.freebase.com/ns/"
str_fault="<http://rdf.freebase.com/ns/people.person.date_of_birth>"
with gzip.open('freebase_douban.gz','rb') as f, gzip.open('output_second_entity.gz','wt') as output1, gzip.open('output_second_3.gz','wt') as output2:
    for line in tqdm(f, desc="Processing lines", unit="line"):
        triplet = line.strip().decode().split('\t')[:3]
        if triplet[0] in second_initial and str_flag in triplet[2] and str_fault not in triplet[1]:
            output1.write(triplet[2]+"\n")
            output2.write(triplet[0]+"\t"+triplet[1]+"\t"+triplet[2]+"\n")

Processing lines: 763line [00:00, 382759.71line/s]


763


Processing lines: 395577070line [1:05:40, 100389.35line/s]


In [121]:
import gzip
from tqdm import tqdm
min_relation = 50  
min_entity = 15
max_entity = 20000

In [122]:
# Assuming douban2fb.txt is a mapping file with movie ID to entity ID
mapping_file = 'douban2fb.txt'

# Create a set to store the entity IDs from douban2fb.txt for faster lookup
entity_ids_list = []
movie_id2fb = {}

with open(mapping_file, 'r') as mapping:
    for line in mapping:
        # Assuming the file format is "movieID\tentityID"
        movie_id, entity_id = line.strip().split()
        entity_id="<http://rdf.freebase.com/ns/" + entity_id+ ">"
        movie_id2fb[movie_id] = entity_id
        entity_ids_list.append(entity_id)

In [123]:
def delete_line(line_to_delete):#line_to_delete是一个list,存储了需要删除的行的行号
    with gzip.open('second_3.gz','rb') as second_3:
    #删除line_to_deiete中的对应行，保存在temp_3中
        with gzip.open('temp_3.gz','wt') as output_temp_3:
            line_num = 0
            i = 0
            num_to_delete = len(line_to_delete)
            for line in tqdm(second_3, desc="Processing lines", unit="line"):
                if(line_num != line_to_delete[i]):
                    output_temp_3.write(line.decode())
                else:
                    if(i < num_to_delete - 1):
                        i = i + 1
                line_num = line_num + 1
    print("task1 successfully!")
    #将temp_3中的内容复制到second_3中
    with gzip.open('temp_3.gz','rb') as output_temp_3:
        with gzip.open('second_3.gz','wt') as output_second_3:
            for line in tqdm(output_temp_3, desc="Processing lines", unit="line"):
                output_second_3.write(line.decode())
    print("task2 successfully!")

In [124]:
import gzip
def update_large(line_to_delete):
    with gzip.open('second_3.gz','rb') as second_3:
        num_entity = {}
        num_relation = {}
        for line in tqdm(second_3, desc="Processing lines", unit="line"):
            triplet = line.decode().strip().split('\t')
            if triplet[2] in num_entity:
                num_entity[triplet[2]] = num_entity[triplet[2]] + 1
            else:
                num_entity[triplet[2]] = 1
            if triplet[0] in num_entity:
                num_entity[triplet[0]] = num_entity[triplet[0]] + 1
            else:
                num_entity[triplet[0]] = 1
            if triplet[1] in num_relation:
                num_relation[triplet[1]] = num_relation[triplet[1]] + 1
            else:
                num_relation[triplet[1]] = 1
        line_number = 0
        second_3.seek(0)
        for line in tqdm(second_3, desc="Processing lines", unit="line"):
            triplet = line.decode().strip().split('\t')
            if num_entity[triplet[2]] > max_entity and triplet[2] not in entity_ids_list:
                line_to_delete.append(line_number)
            elif num_entity[triplet[0]] > max_entity and triplet[0] not in entity_ids_list:
                line_to_delete.append(line_number)
            elif num_relation[triplet[1]] < min_relation:
                line_to_delete.append(line_number)
            line_number = line_number + 1
        print(len(line_to_delete))
        

In [125]:
line_to_delete = []
update_large(line_to_delete)

Processing lines: 104724704line [04:15, 410481.74line/s]
Processing lines: 104724704line [12:42, 137337.57line/s]


103978969


In [126]:
print(len(line_to_delete))

103978969


In [127]:
delete_line(line_to_delete)

Processing lines: 0line [00:00, ?line/s]

Processing lines: 104724704line [01:36, 1085590.38line/s]


task1 successfully!


Processing lines: 745735line [00:02, 341392.33line/s]

task2 successfully!


In [128]:
import gzip
from tqdm import tqdm
second_3 = []
with gzip.open('second_3.gz','rb') as f:
    for line in tqdm(f,desc = "Processing lines",unit = "line"):
        triplet = line.strip().decode().split('\t')[:3]
        second_3.append(triplet)
print(len(second_3))
print(second_3[4])

Processing lines: 745735line [00:08, 85811.85line/s] 

745735
['<http://rdf.freebase.com/ns/m.0kprc8>', '<http://rdf.freebase.com/ns/film.content_rating.film>', '<http://rdf.freebase.com/ns/m.026hy89>']


In [129]:
print(len(second_3))

745735


In [130]:
str = '<http://rdf.freebase.com/ns/m.026pdy>'
#统计str在second_3中出现的次数
count1 = 0
count2 = 0
for i in range(len(second_3)):
    if second_3[i][0] == str:
        count1 = count1 + 1
    elif second_3[i][2] == str:
        count2 = count2 + 1
print(count1)
print(count2)
#打印出与之相关的关系

36
0


In [131]:
min_relation = 50
min_entity = 15
max_entity = 20000
#函数，通过关系小于50更新second_3，通过second_3更新second_jump
def update_second_3_by_relation():
    num_relation = {}#存储每个关系的数量
    num_of_entity = len(second_3)#实体的数量
    for i in range(num_of_entity):
        if second_3[i][1] not in num_relation:
            num_relation[second_3[i][1]] = 1
        else:
            num_relation[second_3[i][1]] = num_relation[second_3[i][1]] + 1
    id_to_delete = []#存储需要删除的三元组的id
    #在second_3中删除关系小于50的三元组,并在second_jump中删除对应的实体
    for i in range(num_of_entity):
        if num_relation[second_3[i][1]] <= min_relation:
            id_to_delete.append(i)
    for i in range(len(id_to_delete)):
        del second_3[id_to_delete[i]-i]
    
def update_second_3_by_entity():
    num_entity = {}#存储每个实体的数量
    num_of_entity = len(second_3)#实体的数量,即三元组的数量
    for i in range(num_of_entity):
        if second_3[i][0] not in num_entity:
            num_entity[second_3[i][0]] = 1
        else:
            num_entity[second_3[i][0]] = num_entity[second_3[i][0]] + 1
        if second_3[i][2] not in num_entity:
            num_entity[second_3[i][2]] = 1
        else:
            num_entity[second_3[i][2]] = num_entity[second_3[i][2]] + 1
    id_to_delete = []#存储需要删除的三元组的id
    for i in range(num_of_entity):
        if num_entity[second_3[i][0]] <= min_entity and second_3[i][0] in entity_ids_list:
            continue
        elif num_entity[second_3[i][0]] <= min_entity and second_3[i][0] not in entity_ids_list:
            id_to_delete.append(i)
        elif num_entity[second_3[i][2]] <= min_entity and second_3[i][2] not in entity_ids_list:
            id_to_delete.append(i)
            
    for i in range(len(id_to_delete)):
        del second_3[id_to_delete[i]-i]

In [132]:
#处理第二跳，删除关系小于50的三元组，删除实体小于15的三元组,轮流执行直至second_3和second_jump不再变化
def process_second_jump():
    i=0
    len_old = len(second_3)
    print(i)
    print(len_old)
    update_second_3_by_entity()
    update_second_3_by_relation()
    while len_old != len(second_3):
        i = i+1
        len_old = len(second_3)
        print(i)
        print(len_old)
        update_second_3_by_entity()
        update_second_3_by_relation()

In [133]:
process_second_jump()

0
745735
1
42777
2
42515
3
42390
4
42387


In [134]:
print(len(second_3))

42387


In [135]:
#判断second_3中的实体是否包含entity_ids_list中的实体
entity_in_second_3 = []
for i in range(len(second_3)):
    entity_in_second_3.append(second_3[i][0])
    entity_in_second_3.append(second_3[i][2])

entity_in_second_3 = set(entity_in_second_3)
print(len(entity_in_second_3))
for i in range(len(entity_ids_list)):
    if entity_ids_list[i] not in entity_in_second_3:
        print(entity_ids_list[i])

1885


In [140]:
#将second_3中的内容重新写入second_3.gz中
import gzip
from tqdm import tqdm
with gzip.open('second_3.gz','wt') as output_second_3:
    for i in tqdm(range(len(second_3)),desc = "Processing lines",unit = "line"):
        output_second_3.write(second_3[i][0]+"\t"+second_3[i][1]+"\t"+second_3[i][2]+"\n")

Processing lines: 100%|██████████| 42387/42387 [00:00<00:00, 282325.38line/s]


In [136]:
#测试代码
#测试正确性
num_entity_second_jump = {}
num_relation_second_jump = {}
num_of_entity = len(second_3)
for i in range(num_of_entity):
    if second_3[i][0] not in num_entity_second_jump:
        num_entity_second_jump[second_3[i][0]] = 1
    else:
        num_entity_second_jump[second_3[i][0]] = num_entity_second_jump[second_3[i][0]] + 1
    if second_3[i][2] not in num_entity_second_jump:
        num_entity_second_jump[second_3[i][2]] = 1
    else:
        num_entity_second_jump[second_3[i][2]] = num_entity_second_jump[second_3[i][2]] + 1
    if second_3[i][1] not in num_relation_second_jump:
        num_relation_second_jump[second_3[i][1]] = 1
    else:
        num_relation_second_jump[second_3[i][1]] = num_relation_second_jump[second_3[i][1]] + 1

#测试正确性,测试数量是否符合要求
min_relation = 50
min_entity = 15
for i in range(num_of_entity):
    if num_entity_second_jump[second_3[i][0]] < min_entity or num_entity_second_jump[second_3[i][0]] > max_entity and second_3[i][0] not in entity_ids_list:
        print("error")
    elif num_entity_second_jump[second_3[i][2]] < min_entity or num_entity_second_jump[second_3[i][2]] > max_entity and second_3[i][2] not in entity_ids_list:
        print("error")
    elif num_relation_second_jump[second_3[i][1]] < min_relation and second_3[i][2] not in entity_ids_list and second_3[i][0] not in entity_ids_list:
        print("error")

print(len(num_entity_second_jump))
print(len(num_relation_second_jump))

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
erro

In [137]:
print(num_relation_second_jump)

{'<http://rdf.freebase.com/ns/film.content_rating.film>': 1314, '<http://rdf.freebase.com/ns/type.object.type>': 2683, '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>': 2628, '<http://rdf.freebase.com/ns/film.film.genre>': 1309, '<http://rdf.freebase.com/ns/freebase.valuenotation.is_reviewed>': 1215, '<http://rdf.freebase.com/ns/media_common.netflix_title.netflix_genres>': 3855, '<http://rdf.freebase.com/ns/common.topic.notable_types>': 588, '<http://rdf.freebase.com/ns/film.film.language>': 1037, '<http://rdf.freebase.com/ns/film.film.rating>': 475, '<http://rdf.freebase.com/ns/film.film.country>': 404, '<http://rdf.freebase.com/ns/freebase.valuenotation.has_value>': 77, '<http://rdf.freebase.com/ns/freebase.valuenotation.has_no_value>': 684, '<http://rdf.freebase.com/ns/film.film.production_companies>': 244, '<http://rdf.freebase.com/ns/film.film.prequel>': 63, '<http://rdf.freebase.com/ns/film.film.sequel>': 73, '<http://rdf.freebase.com/ns/type.type.instance>': 5003, '<http://rd

In [138]:
import csv
#将num_relation_second_jump写入csv文件
with open('num_relation_second_jump.csv','w') as f:
    writer = csv.writer(f)
    for k,v in num_relation_second_jump.items():
        writer.writerow([k,v])
#对于csv文件，进行排序，数值从大到小，并且数值列对齐
import pandas as pd
df = pd.read_csv('num_relation_second_jump.csv',header=None)
df.columns = ['relation','number']
df = df.sort_values(by='number',ascending=False)
df.to_csv('num_relation_second_jump.csv',index=False,header=False)

In [139]:
import csv
#将num_entity_second_jump写入csv文件
with open('num_entity_second_jump.csv','w') as f:
    writer = csv.writer(f)
    for k,v in num_entity_second_jump.items():
        writer.writerow([k,v])